In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import json
import os
import re

# --- 설정 변수 ---
INPUT_CSV_FILE = 'labeled_sentences.csv'
TRAIN_OUTPUT_FILE = 'train_new.jsonl'
VAL_OUTPUT_FILE = 'val_new.jsonl'
TEST_SIZE_RATIO = 0.2  # 검증용 데이터셋 비율 (20%)
RANDOM_STATE_SEED = 42 # 재현성을 위한 시드 설정

# 헤더가 없는 파일이므로 컬럼 이름 임시 지정
COLUMN_NAMES = ['text_input', 'command_string'] 
INSTRUCTION_TEXT = "다음 문장의 감정에 어울리는 H,S,B,Dimmer,CT 값을 예측하세요. JSON으로만 답하세요."

print(f"1. 데이터 로드 및 파싱 준비: {INPUT_CSV_FILE}")

# 1. 데이터 로드: header=None으로 지정하고, names로 컬럼 이름 부여
try:
    # 쉼표(,)를 구분자로 사용하며, 헤더는 없고, 컬럼 이름을 직접 지정합니다.
    df = pd.read_csv(
        INPUT_CSV_FILE, 
        header=None, 
        names=COLUMN_NAMES, 
        sep=','
    )
    total_records = len(df)
    
    print(f"   -> 로드된 총 레코드 수: {total_records}개 (3200개 가정)")
    print(f"   -> 사용된 컬럼 목록: {list(df.columns)}")
    
except FileNotFoundError:
    print(f"에러: 파일을 찾을 수 없습니다! ({INPUT_CSV_FILE})")
    df = None
except Exception as e:
    print(f"데이터 로드 중 예상치 못한 에러 발생: {e}")
    df = None

# ----------------------------------------------------------------------

if df is not None and total_records > 0:
    
    # 2. command_string에서 H, S, B, Dimmer, CT 값을 파싱하여 새 컬럼 생성
    def parse_command(command_str):
        """조명 명령 문자열에서 H, S, B, Dimmer, CT 값을 추출하여 딕셔너리로 반환"""
        
        # HSBCOLOR H,S,B 추출
        hsb_match = re.search(r"HSBCOLOR\s*(\d+),(\d+),(\d+)", command_str)
        # Dimmer D 추출
        dimmer_match = re.search(r"Dimmer\s*(\d+)", command_str)
        # CT C 추출
        ct_match = re.search(r"CT\s*(\d+)", command_str)
        
        if hsb_match and dimmer_match and ct_match:
            try:
                h = int(hsb_match.group(1))
                s = int(hsb_match.group(2))
                b = int(hsb_match.group(3))
                dimmer = int(dimmer_match.group(1))
                ct = int(ct_match.group(1))
                
                return {'h': h, 's': s, 'b': b, 'dimmer': dimmer, 'ct': ct}
            except ValueError:
                # 숫자로 변환할 수 없는 경우 None 반환
                return None
        return None

    # 파싱 함수를 모든 행에 적용
    print("\n2. 명령 문자열에서 H, S, B, Dimmer, CT 값 파싱 중...")
    df['output_dict'] = df['command_string'].apply(parse_command)
    
    # 파싱에 실패한 행(None) 제거 (데이터 클리닝)
    df = df.dropna(subset=['output_dict'])
    cleaned_records = len(df)
    
    if cleaned_records < total_records:
        print(f"   -> 파싱에 성공한 레코드 수: {cleaned_records}개 ({total_records - cleaned_records}개 제거됨)")
    else:
        print(f"   -> 파싱 성공 (총 {cleaned_records}개 레코드 사용)")
        
    if cleaned_records == 0:
        print("   -> 유효한 데이터가 없어 JSONL 파일 생성을 건너뜁니다.")
        exit() # 유효 데이터가 없으면 종료
        
    # 3. 데이터 분할: 80% train / 20% validation
    print(f"\n3. 데이터 분할: Train ({1-TEST_SIZE_RATIO:.0%}) / Validation ({TEST_SIZE_RATIO:.0%})")
    
    train_df, val_df = train_test_split(
        df, 
        test_size=TEST_SIZE_RATIO, 
        random_state=RANDOM_STATE_SEED
    )

    print(f"   -> 훈련 데이터셋 크기: {len(train_df)}개")
    print(f"   -> 검증 데이터셋 크기: {len(val_df)}개")

    # 4. JSONL 형식으로 변환 및 파일 저장 함수
    def save_to_jsonl(dataframe, filename):
        """DataFrame을 요청된 JSONL 형식으로 저장하는 함수"""
        print(f"\n4. 파일 저장: {filename}에 JSONL 형식으로 저장 중...")
        
        records = []
        for index, row in dataframe.iterrows():
            # 최종 JSONL 레코드 형식: { "instruction": "...", "input": "...", "output": { ... } }
            records.append({
                "instruction": INSTRUCTION_TEXT,
                "input": row['text_input'],
                "output": row['output_dict'] # 파싱된 딕셔너리 사용
            })
            
        
        # 파일에 JSONL 형식으로 쓰기
        with open(filename, 'w', encoding='utf-8') as f:
            for record in records:
                # 각 레코드를 JSON 문자열로 변환하고 줄바꿈(\n) 추가
                json_line = json.dumps(record, ensure_ascii=False)
                f.write(json_line + '\n')
                
        print(f"   -> 저장 완료: {filename} (크기: {os.path.getsize(filename) / 1024:.2f} KB)")


    # 훈련 데이터 저장
    save_to_jsonl(train_df, TRAIN_OUTPUT_FILE)
    
    # 검증 데이터 저장
    save_to_jsonl(val_df, VAL_OUTPUT_FILE)

else:
    print("   -> 데이터 로드 실패 또는 데이터가 비어있어 파일 생성을 건너뜁니다.")

1. 데이터 로드 및 파싱 준비: labeled_sentences.csv
   -> 로드된 총 레코드 수: 74201개 (3200개 가정)
   -> 사용된 컬럼 목록: ['text_input', 'command_string']

2. 명령 문자열에서 H, S, B, Dimmer, CT 값 파싱 중...
   -> 파싱에 성공한 레코드 수: 74162개 (39개 제거됨)

3. 데이터 분할: Train (80%) / Validation (20%)
   -> 훈련 데이터셋 크기: 59329개
   -> 검증 데이터셋 크기: 14833개

4. 파일 저장: train_new.jsonl에 JSONL 형식으로 저장 중...
   -> 저장 완료: train_new.jsonl (크기: 16632.91 KB)

4. 파일 저장: val_new.jsonl에 JSONL 형식으로 저장 중...
   -> 저장 완료: val_new.jsonl (크기: 4161.64 KB)
